# Installing required packages

In [37]:
!pip3 install nltk
!pip3 install sklearn
!pip3 install numpy

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
Defaulting to user installation because normal site-packages is not writeable
Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
Defaulting to user installation because normal site-packages is not writeable
Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
Defaulting to user installation because normal site-packages is not writeable


## Downloading nltk packages

In [ ]:
import nltk
nltk.download('wordnet', 'stopwords', 'punkt')

# Importing packages

In [1]:
import json
from math import sqrt, log
import numpy as np
from nltk.stem import PorterStemmer, WordNetLemmatizer
from math import log
from nltk.corpus import stopwords as sw
import time

In [2]:
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier

# Reading data

In [3]:
def get_data(path):    
    '''Reading the json files from passed path'''
    with open(path) as d:
        json_data = json.load(d)
    return json_data

# Data Operations

In [4]:
def get_doc_terms(doc_info):
    '''concatenate all terms (title + body) of the passed doc'''
    body_terms = doc_info['body'].split()
    title_terms = doc_info['title'].split()
    all_terms = body_terms + title_terms
    return all_terms

## Count term frequency over collection

In [5]:
stopwords = sw.words('english')

In [6]:
def get_document_frequency(json_data):
    vectors = {}
    
    lemmatizer = WordNetLemmatizer()
    stemmer = PorterStemmer()
    
    for doc_id, doc in enumerate(json_data):
        terms = get_doc_terms(doc)
        for term in terms:
            if term not in stopwords:
                term = lemmatizer.lemmatize(term)
                if term not in vectors:
                    vectors[term] = []
                if doc_id not in vectors[term]:
                    vectors[term].append(doc_id)
    
    vectors_list = list(vectors.items())
    term_map_index = {}
    
    for i in range(len(vectors_list)):
        term = vectors_list[i][0]
        term_map_index[term] = i
    
    frequencies = []
    for word, freq_list in vectors_list:
        frequencies.append(len(freq_list))
    
    return term_map_index, frequencies

In [7]:
def get_vector(json_data, term_map_index, frequencies, train_docs_no):
    term_no = len(term_map_index)
    data_no = len(json_data)
    vector = np.zeros((data_no, term_no))
        
    for doc_id, doc in enumerate(json_data):
        terms = get_doc_terms(doc)
        doc_term_freq = {}
        
        for term in terms:
            if term not in stopwords:
                if term not in doc_term_freq:
                    doc_term_freq[term] = 0
                doc_term_freq[term] += 1
                
        for term in doc_term_freq:
            if term in term_map_index:
                index = term_map_index[term]
                doc_frequency = frequencies[index]
                tf = doc_term_freq[term]
                idf = log(train_docs_no/doc_frequency)
                vector[doc_id, index] = tf * idf
    
    return vector

In [8]:
def get_category(data):
    '''return list of categories for train and test docs'''
    categories = []
    
    for doc in data:
        category = doc['category']
        categories.append(category)
        
    return categories

## Assigning variables

In [86]:
start_time = time.time()

train_data = get_data('./data/train.json')
test_data = get_data('./data/validation.json')
# print(list(train_data)[:10])


N = len(train_data)

N = N // 12

#randomly pick N data from train
train_data = train_data[:N]
test_data = test_data[:300]

term_map_index, frequencies = get_document_frequency(train_data)

train_categories = get_category(train_data)
test_categories = get_category(test_data)
# print(train_categories)[:10]

#train and test vectors 
train_vector = get_vector(train_data, term_map_index, frequencies, N)
test_vector = get_vector(test_data, term_map_index, frequencies, N)

finish_time = time.time()
print('elapsed time:', finish_time - start_time)

elapsed time: 1.2536721229553223


# kNN (k Nearest Neighbor)

There are two methods used for measuring nearest neighbors: 
1. Cosine Similarity
2. Euclidean Distance

The Following sections would be the implementation of each method. 

## Cosine Similarity

Cosine Similarity = train.test^t / |train|.|test^t|

Since |test^t| is constant, we can ignore its value and don't calculate it. 

## Euclidean Distance

In [21]:
euclidean_distance = None
def calculate_euclidean_distance(train, test):
    '''Euclidean Distance'''
    global euclidean_distance
    x_2 = np.sum(test * test, axis=1)
    test_transpose = test.T
    xy = np.dot(train, test_transpose).T
    y_2 = np.sum(train * train, axis = 1)
    
    #make the test 1-column for evaluation
    x_2 = x_2[:,None]
    result = x_2 + y_2 - 2 * xy
    euclidean_distance = np.sqrt(result)
    return euclidean_distance


In [22]:
cosine_similarity = None
def calculate_cosine_similarity(train, test):
    '''consine_similarity = train.test^t / |train|.|test|'''
    global cosine_similarity
    test_transpose = test.T
    dot_product = np.dot(train, test_transpose).T
    normalization = np.linalg.norm(train, axis=1)
    cosine_similarity = dot_product / normalization
    return cosine_similarity


## kNN 

In [31]:
def knn(doc_id, method, k):
    global cosine_similarity
    global euclidean_distance
    if method == 'cosine':
        similarities = cosine_similarity[doc_id]
        nearest = sorted(range(len(similarities)), key=lambda i: similarities[i], reverse=True)[:k]
    elif method == 'euclidean':
        distances = euclidean_distance[doc_id]
        nearest = sorted(range(len(distances)), key=lambda i: distances[i])[:k]
    categories = {}
    for i in range(len(nearest)):
        category = train_categories[nearest[i]]
        if category not in categories:
          categories[category] = 0
        categories[category] += 1
    
    return max(categories, key=categories.get)
    


# Performance Measure

In [23]:
def get_vector_creation_time(method):
    start_time = time.time()
    
    if method == 'euclidean':
        calculate_euclidean_distance(train_vector, test_vector)
    elif method == 'cosine':
        calculate_cosine_similarity(train_vector, test_vector)
    finish_time = time.time()
    elapsed_time = finish_time - start_time
    return elapsed_time

In [64]:
def get_recall(method, category_no=4, k=5):
    tp, total = [], []
    
    for i in range(category_no):
        tp.append(0)
        total.append(0)
        
    test_no = len(test_data)
    for i in range(test_no):
        real = test_data[i]['category']
        if method == 'bayes':
          predicted = naive_bayes_testing(test_data[i], 1)
        else:
          predicted = knn(i, method, k)
        index = real - 1
        
        if real == predicted:
            tp[index] += 1
            
        total[index] += 1
    
    recall = {}
    
    for i in range(category_no):
        index = i + 1
        recall[index] = tp[i]/total[i]
    
    return recall

In [68]:
def get_precision(method, category_no=4, k=5):
    tp, total = [], []
    
    for i in range(category_no):
        tp.append(0)
        total.append(0)
        
    test_no = len(test_data)
    for i in range(test_no):
        real = test_data[i]['category']
        if method == 'bayes':
          predicted = naive_bayes_testing(test_data[i], 1)
        else:
          predicted = knn(i, method, k)
        index = predicted - 1
        
        if real == predicted:
            tp[real - 1] += 1
            
        total[index] += 1
    
    precision = {}
    
    for i in range(category_no):
        index = i + 1
        precision[index] = tp[i]/total[i]
    
    return precision

In [69]:
def get_accuracy(method, category_no=4, k=5):
    test_no = len(test_data)
    tp = 0
    
    for i in range(test_no):
        if method == 'bayes':
          predicted = naive_bayes_testing(test_data[i], 1)
        else:
          predicted = knn(i, method, k)
        real = test_data[i]['category']
        
        if predicted == real:
            tp += 1
    
    accuracy = tp/test_no
    return accuracy

In [74]:
def get_marco_f1(precision_val, recall_val, category_no=4):
    results = []
    for i in range(1, category_no+1):
        result = 2 * precision_val[i] * recall_val[i] / (precision_val[i] + recall_val[i])
        results.append(result)
    return np.mean(results)

## Euclidean

In [37]:
get_vector_creation_time('euclidean')

48.714179039001465

In [43]:
euclidean_recall = get_recall('euclidean')
euclidean_recall

{1: 0.9613333333333334,
 2: 0.05733333333333333,
 3: 0.144,
 4: 0.3973333333333333}

In [44]:
euclidean_precision = get_precision('euclidean')
euclidean_precision

{1: 0.30486257928118393,
 2: 0.9347826086956522,
 3: 0.8925619834710744,
 4: 0.6367521367521367}

In [41]:
get_accuracy('euclidean')

0.39

In [76]:
get_marco_f1(euclidean_precision, euclidean_recall)

0.32706978241586504

In [48]:
get_vector_creation_time('cosine')

52.60592985153198

In [49]:
cosine_recall = get_recall('cosine')
cosine_recall

{1: 0.832, 2: 0.9186666666666666, 3: 0.7906666666666666, 4: 0.7773333333333333}

In [50]:
cosine_precision = get_precision('cosine')
cosine_precision

{1: 0.8409703504043127,
 2: 0.8699494949494949,
 3: 0.7782152230971129,
 4: 0.828125}

In [51]:
get_accuracy('cosine')

0.8296666666666667

In [75]:
get_marco_f1(cosine_precision, cosine_recall)

0.8291057499806818

# Naive Bayes

In [60]:
n_c, t_c = None, None
lemmatizer = WordNetLemmatizer()
def naive_bayes_training(data, category_no=4):
    global n_c
    global t_c
    term_no = len(term_map_index)
    
    n_c = []
    for i in range(category_no):
        n_c.append(0)
    
    t_c = np.zeros((category_no, term_no))
    
    for doc_id, doc in enumerate(data):
        terms = get_doc_terms(doc)
        category = doc['category']

        for term in terms:
            index = category - 1
            if term not in stopwords:
                term = lemmatizer.lemmatize(term)
                t_c[index, term_map_index[term]] += 1
            n_c[index] += 1
            

In [66]:
def naive_bayes_testing(document, alpha, category_no=4):
    global n_c
    global t_c
    term_no = len(term_map_index)
    scores = []
    
    for i in range(category_no):
        score = log(n_c[i]/N)
        scores.append(score)
    
    sigma = np.sum(t_c, axis=1)
    
    for i in range(category_no):
        terms = get_doc_terms(document)
        for term in terms:
            if term not in stopwords:
                term = lemmatizer.lemmatize(term)
                if term in term_map_index:
                    fraction = log((t_c[i, term_map_index[term]] + alpha) / (sigma[i] + term_no))
                    scores[i] += fraction
    predicted = np.argmax(scores) + 1

    return predicted


## Performance Measure

In [61]:
naive_bayes_training(train_data)

In [70]:
bayes_recall = get_recall('bayes')
bayes_recall

{1: 0.88, 2: 0.9746666666666667, 3: 0.8546666666666667, 4: 0.856}

In [71]:
bayes_precision = get_precision('bayes')
bayes_precision

{1: 0.9078404401650619,
 2: 0.9300254452926209,
 3: 0.8627187079407806,
 4: 0.8629032258064516}

In [72]:
get_accuracy('bayes')

0.8913333333333333

In [77]:
get_marco_f1(bayes_precision, bayes_recall)

0.8909094829335988

# Random Forest

In [87]:
start_time = time.time()
clf = svm.SVC(kernel='linear', C=1)
clf.fit(train_vector, train_categories)
finish_time = time.time()
elapsed_time = finish_time - start_time

In [88]:
elapsed_time

138.9300298690796

In [89]:
clf.score(test_vector, test_categories)

0.8433333333333334

In [90]:
start_RandomForestClassifiermForestClassifiermax_depth=Classifier= time.time()
clf = RandomForestClassifier(n_estimators=10, max_depth=None, min_samples_split=2, random_state=0)
clf.fit(train_vector, train_categories)
finish_time = time.time()
elapsed_time = finish_time - start_time

In [92]:
clf.score(test_vector, test_categories)

0.6733333333333333